# Set Up

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
import sys
from google.colab import drive
import os

drive.mount('/content/drive')
drive_path = os.path.join('drive', 'MyDrive', 'Colab Notebooks', 'Dacon', 'SentenceOrder') # 다른 드라이브에서 사용할 경우, 이 부분만 적절히 수정

if drive_path not in sys.path:
  sys.path.append(drive_path)
from utility import *

SEED=42
CONFIG_PATH = drive_path + '/config.yaml'

set_all_seed(SEED)
config = load_config(CONFIG_PATH)

In [ ]:
test = pd.read_csv(drive_path + '/dataset/test.csv').drop(columns = 'ID')

# Load the Model and the Tokenizer

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
  model_name = config['fine_tuned_model_name'], # JuyeopDang/Qwen-3-14B-Sentence-Ordering
  max_seq_length = 512,
  load_in_4bit = True,
)

model.eval()
FastLanguageModel.for_inference(model)

In [ ]:
inference_message = [
    {"role": "system", "content": "You are an expert at understanding the logical flow of sentences. Your task is to arrange four given Korean sentences into a coherent and natural paragraph. Output the ordered sequence of sentence indices, separated by commas. /no_think"},
    {"role": "user", "content": "Provided Sentences:\n0. {sentence_0}\n1. {sentence_1}\n2. {sentence_2}\n3. {sentence_3}"}
]

inference_chat_template = tokenizer.apply_chat_template(inference_message, tokenize=False, add_generation_prompt=True, enable_thinking=False)
print(inference_chat_template)

# Evaluate

In [ ]:
def evaluation(model, tokenizer, row, chat_template):
  # 하나의 테스트 데이터를 모델이 추론

  input = tokenizer(
      [ formatting_prompts(row, chat_template) ],
      return_tensors="pt"
  ).to("cuda")

  outputs = model.generate(
      input_ids=input.input_ids,
      attention_mask=input.attention_mask,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=32,
      use_cache=True,
      do_sample=False,
      temperature=None,
      top_p=None,
      top_k=None,
      min_p=None,
      num_beams=1,
  )
  response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  ans = list(map(int, response[0][-10:].split(','))) # 다른 텍스트를 전부 제외한, 순수 정답만 추출한 후 리스트로 변환

  return ans

In [ ]:
sample_submission = pd.read_csv(drive_path + '/dataset/sample_submission.csv')

for i, row in tqdm(test.iterrows()):
  ans = evaluation(model, tokenizer, row, inference_chat_template)
  for j in range(4):
    sample_submission.iloc[i, 1 + j] = ans[j]
sample_submission.to_csv(drive_path + f'/prediction/final_submission.csv')